In [ ]:
! pip install kaggle

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! mkdir ~/.kaggle

In [ ]:

! cp /content/drive/MyDrive/Stat_Docs/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
# Change permission
! chmod 600 ~/.kaggle/kaggle.json

### Dataset Download 

In [ ]:
! kaggle datasets download -d awsaf49/cbis-ddsm-breast-cancer-image-dataset

In [ ]:
! unzip 

In [ ]:
# check disk usage
! df -h

In [ ]:
import os
import PIL
import cv2
import uuid
import shutil
import random
import glob as gb
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from tqdm import tqdm  # Progress bar
from scipy.special import gamma

from keras.optimizers import *
from keras.regularizers import l1_l2
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers import GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.layers import Conv2D, MaxPool2D, BatchNormalization

from tensorflow.keras.metrics import *
from tensorflow.keras.callbacks import *

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Load data 
# check correct file path on colab 
calc_train = pd.read_csv('/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_train_set.csv')
calc_test = pd.read_csv('/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/calc_case_description_test_set.csv')
mass_train = pd.read_csv('/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_train_set.csv')
mass_test = pd.read_csv('/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/mass_case_description_test_set.csv')
dicom_df = pd.read_csv('/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/csv/dicom_info.csv')

In [ ]:
def replace_path(sample, old_path, new_path):
    return sample.replace(old_path, new_path, regex=True)


# if fxn above doesn't work try pd dataframe-type 
# to get 
# import pandas as pd

# # Sample input (pandas Series)
# data = pd.Series(["/old/path/file1.jpg", "/old/path/file2.jpg", "/other/path/file3.jpg"])

# # Replace "/old/path" with "/new/path"
# updated_data = replace_path(data, "/old/path", "/new/path")

# print(updated_data)

# 0    /new/path/file1.jpg
# 1    /new/path/file2.jpg
# 2    /other/path/file3.jpg
# dtype: object


In [ ]:
def plot_smaples(sample, row=15, col=15):
    plt.figure(figsize=(row, col))
    for i, file in enumerate(sample[0:5]):
#         PIL.Image.open(file): Opens the image file specified by file using the Python Imaging Library (PIL).
# convert("L"): Converts the image to grayscale ("L" mode), which reduces the color channels to shades of gray.
        cropped_images_show = PIL.Image.open(file)
        gray_img= cropped_images_show.convert("L")
        plt.subplot(1,5,i+1)
        plt.imshow(gray_img, cmap='gray')
        plt.axis('off')
    plt.show()

In [ ]:
cropped_images = dicom_df[dicom_df.SeriesDescription=="cropped images"].image_path
full_mammogram = dicom_df[dicom_df.SeriesDescription=="full mammogram images"].image_path
roi_mask = dicom_df[dicom_df.SeriesDescription=="ROI mask images"].image_path

In [ ]:
# Replace the path for cropped_images to the correct directory -- check on colab.
correct_dir = "../input/cbis-ddsm-breast-cancer-image-dataset/jpeg"
cropped_images = replace_path(cropped_images, "CBIS-DDSM/jpeg", correct_dir)
print('Cropped Images paths:')
print(cropped_images.iloc[0]) # Print to ensure everything looks correct.

In [ ]:
# Replace the path for full_mammogram images to the correct directory.
full_mammogram = replace_path(full_mammogram, "CBIS-DDSM/jpeg", correct_dir)
print('\nFull mammo Images paths:')
print(full_mammogram.iloc[0]) # Print to ensure everything looks correct.

In [ ]:
# Replace the path for roi_mask images to the correct directory.
roi_mask = replace_path(roi_mask, "CBIS-DDSM/jpeg", correct_dir)
print('\nROI Mask Images paths:')
print(roi_mask.iloc[0]) # Print to ensure everything looks correct.

In [ ]:
# find length of each dataset and ensure it matches -- all pictures are implemented as expected
def get_image_file_name(data, new_dict):

    for dicom in data:
        key = dicom.split('/')[4]
        new_dict[key] = dicom
    print(f"the length of dataset ==> {len(new_dict.keys())}")

cropped_images_dict = dict()
full_mammo_dict = dict()
roi_img_dict = dict()

get_image_file_name(cropped_images, cropped_images_dict)
get_image_file_name(full_mammogram, full_mammo_dict)
get_image_file_name(roi_mask, roi_img_dict)


In [ ]:
# correct the dicom paths to correct image paths
def fix_image_path(data):
    """Correct dicom paths to correct image paths."""
    for indx, image in enumerate(data.values):
        # Extract third component (index 2) of the path stored in column 11 (image[11]), 
        #assign resulting string to img_name.
        img_name = image[11].split('/')[2]

        # Check if img_name exists as a key in the dictionary full_mammo_dict.
        # If exists, updates the value in column 11 of the current row (data.iloc[indx, 11]) with the corresponding value from the dictionary.
        if img_name in full_mammo_dict:
            data.iloc[indx, 11] = full_mammo_dict[img_name]
        else:
            data.iloc[indx, 11] = None
        

#         Extracts the third component (index 2) of the path stored in column 12 (image[12]).
# Assigns the result to img_name.
        img_name = image[12].split('/')[2]

        # Checks if img_name exists as a key in the dictionary cropped_images_dict.If it exists, updates the value in column 12 of the current row (data.iloc[indx, 12]) with the corresponding value from the dictionary.
        if img_name in cropped_images_dict:
            data.iloc[indx, 12] = cropped_images_dict[img_name]
        else:
            data.iloc[indx, 11] = None


        img_name = image[13].split('/')[2]
        if img_name in roi_img_dict:
            data.iloc[indx, 13] = roi_img_dict[img_name]

        else:
            data.iloc[indx, 13] = None

In [ ]:
# implement the function to fix image file paths for mass_train, mass_test, calc_train and calc_test
fix_image_path(mass_train)

In [ ]:
mass_train = mass_train.rename(columns={'left or right breast': 'left_or_right_breast',
                                        'image view': 'image_view',
                                        'abnormality id': 'abnormality_id',
                                        'abnormality type': 'abnormality_type',
                                        'mass shape': 'mass_shape',
                                        'mass margins': 'mass_margins',
                                        'image file path': 'image_file_path',
                                        'cropped image file path': 'cropped_image_file_path',
                                        'ROI mask file path': 'ROI_mask_file_path'})
mass_train.head(5)

In [ ]:
mass_train.unique()

In [ ]:
# fix image mass test
fix_image_path(mass_test)

In [ ]:
mass_test = mass_test.rename(columns={'left or right breast': 'left_or_right_breast',
                                      'image view': 'image_view',
                                      'abnormality id': 'abnormality_id',
                                      'abnormality type': 'abnormality_type',
                                      'mass shape': 'mass_shape',
                                      'mass margins': 'mass_margins',
                                      'image file path': 'image_file_path',
                                      'cropped image file path': 'cropped_image_file_path',
                                      'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
mass_test.head()

In [ ]:
calc_train = calc_train.rename(columns={'left or right breast': 'left_or_right_breast',
                                        'image view': 'image_view',
                                        'abnormality id': 'abnormality_id',
                                        'abnormality type': 'abnormality_type',
                                        'mass shape': 'mass_shape',
                                        'mass margins': 'mass_margins',
                                        'image file path': 'image_file_path',
                                        'cropped image file path': 'cropped_image_file_path',
                                        'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
calc_train.head()

In [ ]:
fix_image_path(calc_train)

In [ ]:
calc_test = calc_test.rename(columns={'left or right breast': 'left_or_right_breast',
                                      'image view': 'image_view',
                                      'abnormality id': 'abnormality_id',
                                      'abnormality type': 'abnormality_type',
                                      'mass shape': 'mass_shape',
                                      'mass margins': 'mass_margins',
                                      'image file path': 'image_file_path',
                                      'cropped image file path': 'cropped_image_file_path',
                                      'ROI mask file path': 'ROI_mask_file_path'})
# view renamed columns
calc_test.head()

In [ ]:
fix_image_path(calc_test)

In [1]:
# Display images function 
def display_images(dataset, column, number):
    fig, axes = plt.subplots(1, number, figsize=(15,5))
    for index, (i, row) in enumerate(dataset.head(number).iterrows()):
                image_path = row[column]
                # check for valid path
                if image_path is None or not os.path.exists(image_path):
                        # print(f"File not found or invalid path: {image_path}")
                        continue
                # read image (BGR format)
                image = cv2.imread(image_path)
                # handle image read errors
                if image is None:
                        # print(f"Eror reading image: {image_path})
                        continue
                # convert bgr to rgb
                if len(image.shape) == 3 and image.shape[2] == 3:
                        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # assign subplot index to ax 
                ax = axes[index]
                ax.imshow(image, cmap='gray' if len(image.shape) == 2 else None)
                ax.set_title(f"{row['pathology']}")
                ax.axis('off')

                # print image shape 
                print(np.array(image).shape)

                # finalize and display
                plt.tight_layout()
                plt.show()


                        


SyntaxError: expected ':' (1798544137.py, line 4)

In [ ]:
print('Full Mammograms:\n')
display_images(mass_train, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(mass_train, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(mass_train, 'ROI_mask_file_path', 5)

In [ ]:
print('Full Mammograms:\n')
display_images(mass_test, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(mass_test, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(mass_test, 'ROI_mask_file_path', 5)

In [ ]:
print('Full Mammograms:\n')
display_images(calc_train, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(calc_train, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(calc_train, 'ROI_mask_file_path', 5)

In [ ]:
print('Full Mammograms:\n')
display_images(calc_test, 'image_file_path', 5)
print('Cropped Mammograms:\n')
display_images(calc_test, 'cropped_image_file_path', 5)
print('ROI_mask:\n')
display_images(calc_test, 'ROI_mask_file_path', 5)

In [ ]:
# combine both datasets to one 
full_dataset = pd.concat([calc_train, calc_test], axis = 0)

In [ ]:
len(full_dataset)

In [ ]:
# treat  BENIGN_WITHOUT_CALLBACK as 0
class_mapper = {'MALIGNANT': 1, 'BENIGN': 0, 'BENIGN_WITHOUT_CALLBACK': 0}

In [ ]:
target_size = (224, 224, 3)

# apply class mapper to pathology column 
full_dataset['labels'] = full_dataset['pathology'].replace(class_mapper).infer_objects(copy=False)

full_images = np.array(full_dataset[full_dataset["image_file_path"].notna()]['image_file_path'].tolist())

full_labels = np.array(full_dataset[full_dataset['image_file_path'].notna()]['labels'].tolist())

In [ ]:
len(ful_images)

In [ ]:
# convert full_labels from numpy arra to pandas as series
full_labels_series = pd.Series(full_labels)

# count occurences of ech class
label_counts = full_labels_series.value.counts()

benign_count = label_counts.get(0, 0)
malignant_count = label_counts.get(1, 0)

print(f"Benign IMages: {benign_count}")
print(f"Malignant IMages: {malignant_count}")

In [ ]:
num_classes = len(full_dataset['labels'].unique())
num_classes

In [ ]:
class_names = ['Benign', 'Malignant']

In [ ]:
# check distribution of labels
label_counts = full_dataset['labels'].value_counts()
print(label_counts)

In [ ]:
# Data Augmentation 
def augment_image(image):
    # Apply data augmentation using tf.image functions
    image = tf.image.random_flip_left_right(image)
#     image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.3)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.random_saturation(image, lower=0.8, upper=1.2)
    return image

# Function to resize image to (224, 224, 3)
def resize_image(image_tensor):
    return tf.image.resize(image_tensor, [224, 224])

# Function to balance classes by augmenting images
def copy_images_with_unique_filenames(images, labels, source, destination, target_count=None):
    """
    Copy images from source to destination in subfolders '0' and '1',
    ensuring unique filenames and applying data augmentation and balancing.
    """
    benign_images = 0
    malignant_images = 0
    skipped_images = []

    # Create the destination subfolders '0' and '1'
    category_dest_dir_zero = os.path.join(destination, '0')
    os.makedirs(category_dest_dir_zero, exist_ok=True)

    category_dest_dir_one = os.path.join(destination, '1')
    os.makedirs(category_dest_dir_one, exist_ok=True)

    benign_images_list = []
    malignant_images_list = []

    for i, (image, label) in enumerate(zip(images, labels)):
#         img_name = data_frame.REFNUM[i]
#         abs_path = os.path.join(source, img_name + '.pgm')

        if os.path.exists(image):
            try:
                # Generate a unique filename
                filename = os.path.basename(image)
                unique_filename = f"{uuid.uuid4().hex}_{filename}"
        
                # Open the image using PIL
                with Image.open(image) as img:
                    # Convert the image to RGB mode (for saving as JPEG)
                    img = img.convert('RGB')
                    # Augment the image (convert it to a Tensor first)
                    img_tensor = tf.convert_to_tensor(img)
                    # Resize the image to (224, 224, 3)
                    resized_img_tensor = resize_image(img_tensor)
                    augmented_image_tensor = augment_image(resized_img_tensor)
                    # Convert Tensor back to PIL image for saving
                    augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image_tensor)

                    if label == 0:
                        benign_images_list.append(unique_filename)
                        dest_path = os.path.join(category_dest_dir_zero, unique_filename)
#                         augmented_image.save(dest_path, 'JPEG')
                        augmented_image.save(dest_path, 'JPEG')
                        benign_images += 1

                    elif label == 1:
                        malignant_images_list.append(unique_filename)
                        dest_path = os.path.join(category_dest_dir_one, unique_filename)
#                         augmented_image.save(dest_path, 'JPEG')
                        augmented_image.save(dest_path, 'JPEG')
                        malignant_images += 1
                        
#                 del img, img_tensor, resized_img_tensor, augmented_image_tensor, augmented_image
#                 gc.collect()
            except Exception as e:
                print(f"Error copying image {image}: {e}")
                skipped_images.append(image)
        else:
            print(f"Image not found: {image}")
            skipped_images.append(image)

    # If balancing is needed, duplicate/augment images from the smaller class
    benign_count = len(benign_images_list)
    malignant_count = len(malignant_images_list)

    if benign_count < malignant_count:
#         augment_and_save_images(benign_images_list, category_dest_dir_zero, target_count - benign_count)
        augment_and_save_images(benign_images_list, category_dest_dir_zero, malignant_count - benign_count)

    elif malignant_count < benign_count:
        augment_and_save_images(malignant_images_list, category_dest_dir_one, benign_count - malignant_count)

    augment_and_save_images(benign_images_list, category_dest_dir_zero, target_count)
    augment_and_save_images(malignant_images_list, category_dest_dir_one, target_count)

    print(f"\nCopying complete.")
    print(f"Benign images copied (label 0): {benign_images}")
    print(f"Benign count (label 0): {benign_count}")
    print(f"Malignant images copied (label 1): {malignant_images}")
    print(f"Malignant count (label 1): {malignant_count}")
    print(f"Total skipped images: {len(skipped_images)}")
    if skipped_images:
        print("Skipped images:")
        for img in skipped_images:
            print(img)
            

# Function to augment and save images to balance the dataset
def augment_and_save_images(images_list, destination_dir, num_augments):
    """
    Augment and save images to balance the dataset.
    """
    for i in range(num_augments):
        img_name = random.choice(images_list)
        abs_path = os.path.join(destination_dir, img_name)

        try:
            with Image.open(abs_path) as img:
                img = img.convert('RGB')
                # Augment the image
                img_tensor = tf.convert_to_tensor(img)
                # Resize the image
#                 resized_img_tensor = resize_image(img_tensor)
                augmented_image_tensor = augment_image(img_tensor)
                # Convert Tensor back to PIL image for saving
                augmented_image = tf.keras.preprocessing.image.array_to_img(augmented_image_tensor)
                # Remove the original extension from img_name 1-285.jpg --> 1-285
                img_name_without_ext = os.path.splitext(img_name)[0]
                # Save augmented image with a unique name
                augmented_image.save(os.path.join(destination_dir, img_name_without_ext + f'_aug{i}.jpg'), 'JPEG')
            
        except Exception as e:
            print(f"Error augmenting image {abs_path}: {e}")

source_dir = "/kaggle/input/cbis-ddsm-breast-cancer-image-dataset/jpeg"
destination_dir = "/kaggle/working/merged_images"

# target_count=0 meaning no Augmentation, There's just Data-Balance
target_count = (len(full_labels) * 3) - len(full_labels)
copy_images_with_unique_filenames(full_images, full_labels, source_dir, destination_dir, target_count)

In [ ]:
# check number of images in each class folder after merging 
zero_class_count = len(os.listdir("kaggle/working/merged_images/0"))
one_class_count = len(os.listdir("kaggle.working/merged_images/1"))

print(f"Number of images in class 0: {zero_class_count}")
print(f"Number of images in class 1: {one_class_count}")

In [ ]:
import tensorflow as tf

data_dir = '/kaggle/working/merged_images'  # Update with the dataset path

# Create a dataset for the entire data to use for split
full_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    labels='inferred',
    label_mode='categorical',
    # image_size=(224, 224),
    image_size=(224, 224),
    seed=50,
    shuffle=True,
    batch_size=13
)
# Calculate the total number of samples
total_samples = tf.data.experimental.cardinality(full_dataset).numpy()

train_size = int(0.8 * total_samples)                 # 70% for training
val_size   = int(0.15 * total_samples)                # 20% for validation
test_size = total_samples - train_size - val_size     # 10% for testing

# Create train, validation, and test datasets
train_dataset       = full_dataset.take(train_size)
validation_dataset  = full_dataset.skip(train_size).take(val_size)
test_dataset        = full_dataset.skip(train_size + val_size)

train_dataset      = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset       = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Print the number of samples in each dataset
print(f"Train samples:      {train_size}     batches(13) ==> {train_size*13}")
print(f"Validation samples: {val_size}       batches(13) ==> {val_size*13}")
print(f"Test samples:       {test_size}      batches(13) ==> {test_size*13}")

### Model Builds

In [ ]:
from tensorflow.keras.applications import ResNet50

def try_model():
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze all layers initially
    for layer in base_model.layers:
        layer.trainable = False

    # Calculate the index to start unfreezing layers
    from_index = int(np.round((len(base_model.layers) - 1) * (1.0 - 50.0 / 100.0)))

    # Unfreeze layers from the calculated index onwards
    for layer in base_model.layers[from_index:]:
        layer.trainable = True

    # Add custom layers on top (Upper-Layers)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(2, activation='softmax')(x)  # Assuming binary classification

    model = Model(inputs=base_model.input, outputs=predictions)
    
    # # Clear the base model from memory if needed (optional)
    # del model_dict, base_model, from_index, x, predictions;    gc.collect()
    return model

In [ ]:
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.optimizers import Adam

trymodel = try_model()

trymodel.compile(optimizer=Adam(learning_rate=1e-4),
                      loss='categorical_crossentropy',
                      metrics=['accuracy', Precision(name='precision'), Recall(name='recall')])  # Compile the model
trymodel.summary()

In [ ]:
history = trymodel.fit(
            train_dataset,
            validation_data=validation_dataset,
            batch_size=13,
            epochs=7
        )

In [ ]:
# 2nd Model
# Define the second model with adjusted hyperparameters
def try_model_v2():
    base_model_v2 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze all layers initially
    for layer in base_model_v2.layers:
        layer.trainable = False

    # Unfreeze 60% of layers instead of 50%
    from_index_v2 = int(np.round((len(base_model_v2.layers) - 1) * (1.0 - 60.0 / 100.0)))

    # Unfreeze layers from the calculated index onwards
    for layer in base_model_v2.layers[from_index_v2:]:
        layer.trainable = True

    # Add custom layers on top (Upper-Layers)
    x_v2 = base_model_v2.output
    x_v2 = GlobalAveragePooling2D()(x_v2)
    x_v2 = Dense(512, activation='relu')(x_v2)  # Reduced Dense layer size from 1024 to 512
    x_v2 = Dropout(0.3)(x_v2)  # Lowered dropout rate from 0.5 to 0.3
    predictions_v2 = Dense(2, activation='softmax')(x_v2)  # Assuming binary classification

    model_v2 = Model(inputs=base_model_v2.input, outputs=predictions_v2)
    
    return model_v2

# Create and compile the second model
trymodel_v2 = try_model_v2()
trymodel_v2.compile(optimizer=Adam(learning_rate=5e-5),  # Adjusted learning rate to 5e-5
                    loss='categorical_crossentropy',
                    metrics=['accuracy', Precision(name='precision_v2'), Recall(name='recall_v2')])

In [ ]:
# Train the second model
history_v2 = trymodel_v2.fit(
            train_dataset,
            validation_data=validation_dataset,
            batch_size=13,
            epochs=7
        )

# Summarize the second model
trymodel_v2.summary()

In [ ]:
# Third Model
# Define the third model with different hyperparameters
def try_model_v3():
    base_model_v3 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

    # Freeze all layers initially
    for layer in base_model_v3.layers:
        layer.trainable = False

    # Unfreeze 70% of layers (more layers unfreezed compared to v1 and v2)
    from_index_v3 = int(np.round((len(base_model_v3.layers) - 1) * (1.0 - 70.0 / 100.0)))

    # Unfreeze layers from the calculated index onwards
    for layer in base_model_v3.layers[from_index_v3:]:
        layer.trainable = True

    # Add custom layers on top (Upper-Layers)
    x_v3 = base_model_v3.output
    x_v3 = GlobalAveragePooling2D()(x_v3)
    x_v3 = Dense(1024, activation='relu')(x_v3)  # Return Dense layer size to 1024
    x_v3 = Dropout(0.4)(x_v3)  # Moderate dropout rate (between v1 and v2)
    predictions_v3 = Dense(2, activation='softmax')(x_v3)  # Assuming binary classification

    model_v3 = Model(inputs=base_model_v3.input, outputs=predictions_v3)
    
    return model_v3

# Create and compile the third model
trymodel_v3 = try_model_v3()
trymodel_v3.compile(optimizer=RMSprop(learning_rate=1e-4),  # Change optimizer to RMSprop and learning rate back to 1e-4
                    loss='categorical_crossentropy',
                    metrics=['accuracy', Precision(name='precision_v3'), Recall(name='recall_v3')])


In [ ]:
# Train the third model
history_v3 = trymodel_v3.fit(
            train_dataset,
            validation_data=validation_dataset,
            batch_size=13,
            epochs=7
        )

# Summarize the third model
trymodel_v3.summary()

In [ ]:
# Model Comaprison
import matplotlib.pyplot as plt

# Plot accuracy comparison
plt.figure(figsize=(12, 8))

# Accuracy
plt.subplot(2, 2, 1)
plt.plot(history.history['accuracy'], label='Model 1 Accuracy')
plt.plot(history_v2.history['accuracy'], label='Model 2 Accuracy')
plt.plot(history_v3.history['accuracy'], label='Model 3 Accuracy')
plt.title('Training Accuracy')
plt.legend()

# Validation Accuracy
plt.subplot(2, 2, 2)
plt.plot(history.history['val_accuracy'], label='Model 1 Val Accuracy')
plt.plot(history_v2.history['val_accuracy'], label='Model 2 Val Accuracy')
plt.plot(history_v3.history['val_accuracy'], label='Model 3 Val Accuracy')
plt.title('Validation Accuracy')
plt.legend()

# Loss
plt.subplot(2, 2, 3)
plt.plot(history.history['loss'], label='Model 1 Loss')
plt.plot(history_v2.history['loss'], label='Model 2 Loss')
plt.plot(history_v3.history['loss'], label='Model 3 Loss')
plt.title('Training Loss')
plt.legend()

# Validation Loss
plt.subplot(2, 2, 4)
plt.plot(history.history['val_loss'], label='Model 1 Val Loss')
plt.plot(history_v2.history['val_loss'], label='Model 2 Val Loss')
plt.plot(history_v3.history['val_loss'], label='Model 3 Val Loss')
plt.title('Validation Loss')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Plot Precision
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['precision'], label='Model 1 Precision')
plt.plot(history_v2.history['precision_v2'], label='Model 2 Precision')
plt.plot(history_v3.history['precision_v3'], label='Model 3 Precision')
plt.title('Training Precision')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['val_precision'], label='Model 1 Val Precision')
plt.plot(history_v2.history['val_precision_v2'], label='Model 2 Val Precision')
plt.plot(history_v3.history['val_precision_v3'], label='Model 3 Val Precision')
plt.title('Validation Precision')
plt.legend()

plt.tight_layout()
plt.show()

# Plot Recall
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(history.history['recall'], label='Model 1 Recall')
plt.plot(history_v2.history['recall_v2'], label='Model 2 Recall')
plt.plot(history_v3.history['recall_v3'], label='Model 3 Recall')
plt.title('Training Recall')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['val_recall'], label='Model 1 Val Recall')
plt.plot(history_v2.history['val_recall_v2'], label='Model 2 Val Recall')
plt.plot(history_v3.history['val_recall_v3'], label='Model 3 Val Recall')
plt.title('Validation Recall')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Extract metrics for each model's history
metrics_data = {
    'Model': [],
    'Epoch': [],
    'Training Accuracy': [],
    'Validation Accuracy': [],
    'Training Loss': [],
    'Validation Loss': []
}

# Model 1 (history)
for epoch in range(len(history.history['accuracy'])):
    metrics_data['Model'].append('Model 1')
    metrics_data['Epoch'].append(epoch + 1)
    metrics_data['Training Accuracy'].append(history.history['accuracy'][epoch])
    metrics_data['Validation Accuracy'].append(history.history['val_accuracy'][epoch])
    metrics_data['Training Loss'].append(history.history['loss'][epoch])
    metrics_data['Validation Loss'].append(history.history['val_loss'][epoch])

# Model 2 (history_v2)
for epoch in range(len(history_v2.history['accuracy'])):
    metrics_data['Model'].append('Model 2')
    metrics_data['Epoch'].append(epoch + 1)
    metrics_data['Training Accuracy'].append(history_v2.history['accuracy'][epoch])
    metrics_data['Validation Accuracy'].append(history_v2.history['val_accuracy'][epoch])
    metrics_data['Training Loss'].append(history_v2.history['loss'][epoch])
    metrics_data['Validation Loss'].append(history_v2.history['val_loss'][epoch])

# Model 3 (history_v3)
for epoch in range(len(history_v3.history['accuracy'])):
    metrics_data['Model'].append('Model 3')
    metrics_data['Epoch'].append(epoch + 1)
    metrics_data['Training Accuracy'].append(history_v3.history['accuracy'][epoch])
    metrics_data['Validation Accuracy'].append(history_v3.history['val_accuracy'][epoch])
    metrics_data['Training Loss'].append(history_v3.history['loss'][epoch])
    metrics_data['Validation Loss'].append(history_v3.history['val_loss'][epoch])

# Create a DataFrame from the metrics data
metrics_df = pd.DataFrame(metrics_data)

# Display the metrics table
print(metrics_df)

# Save the metrics table to a CSV file
metrics_df.to_csv('model_metrics.csv', index=False)